In [1]:
import csv
import random
from autocorrect import Speller
import schedule
import time
import config
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from random import randint
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from matplotlib import pyplot
import numpy as np
import pickle
from sklearn import datasets # sklearn comes with some toy datasets to practise
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from matplotlib import pyplot
from sklearn.metrics import silhouette_score
import pickle
import config

In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials


#Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))

# The "sp" variable has two useful funtions:
# The first usefull function is:
# .search(q='',limit=n)
# .search(q="track:"+song_name+" artist:"+artist_name,limit=5) to restrict to a song name and artist.
# Where the "q" keyword is the query you want to perform on spotify: song_name, artist,...
# while The "limit" keyword will limit the number of returned results.
#
# The second usefull function is:
# .audio_features([URL|URI|ID])
# which returns some 'features of the song', that after cleanup, we can use in order to characterize a song.

results = sp.search(q="Lose yourself",limit=3,market="GB")
results
#json_results = json.dumps(results, ensure_ascii=True)
#json_results
#results['tracks']['items'][0]['external_urls']["spotify"]

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Lose+yourself&type=track&market=GB&offset=0&limit=3',
  'items': [{'album': {'album_type': 'single',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7dGJo4pcD2V6oG8kP0tJRR'},
       'href': 'https://api.spotify.com/v1/artists/7dGJo4pcD2V6oG8kP0tJRR',
       'id': '7dGJo4pcD2V6oG8kP0tJRR',
       'name': 'Eminem',
       'type': 'artist',
       'uri': 'spotify:artist:7dGJo4pcD2V6oG8kP0tJRR'}],
     'external_urls': {'spotify': 'https://open.spotify.com/album/1rfORa9iYmocEsnnZGMVC4'},
     'href': 'https://api.spotify.com/v1/albums/1rfORa9iYmocEsnnZGMVC4',
     'id': '1rfORa9iYmocEsnnZGMVC4',
     'images': [{'height': 640,
       'url': 'https://i.scdn.co/image/ab67616d0000b273b6ef2ebd34efb08cb76f6eec',
       'width': 640},
      {'height': 300,
       'url': 'https://i.scdn.co/image/ab67616d00001e02b6ef2ebd34efb08cb76f6eec',
       'width': 300},
      {'height': 64,
       'url': 'https://i.

In [85]:
feat_df = pd.read_csv("song_feat.csv")
song_dets_clust = pd.read_csv("song_dets_clust.csv", index_col=0)
song_dets_clust.head()

,song_name,song_id,song_artist,album_name,cluster
0,Unholy (feat. Kim Petras),3nqQXoyQOWXiESFLlDF1hG,"['Sam Smith', 'Kim Petras']",Unholy (feat. Kim Petras),4
1,Anti-Hero,0V3wPSX9ygBnCm8psDIegu,['Taylor Swift'],Midnights,4
2,Rich Flex,1bDbXMyjaUIooNwFE9wn0N,"['Drake', '21 Savage']",Her Loss,2
3,As It Was,4Dvkj6JhhA12EX05fT7y2e,['Harry Styles'],Harry's House,1
4,I'm Good (Blue),4uUG5RXrOk84mYEfFvj3cK,"['David Guetta', 'Bebe Rexha']",I'm Good (Blue),1


In [21]:
def load(filename = "kmeans_8.pickle"): 
    try: 
        with open(filename, "rb") as f: 
            return pickle.load(f) 
        
    except FileNotFoundError: 
        print("File not found!")
kmeans = load()

In [15]:
scaler2 = load("scaler.pickle")

#updateing top 100 song list
def job(t):
    print ("I'm working...", t)
    %run ./Musik_recommender.ipynb
    return
schedule.every().day.at("10:03").do(job,'It is 10:03')

while True:
    schedule.run_pending()
    time.sleep(60)

In [7]:
top_songs = pd.read_csv("top_songs.csv")

In [8]:
#function for user input
def song_input():
    user_answer = input("What's you fauvorite song?")
    spell = Speller(lang='en')
    corr_input = spell(user_answer)
    return corr_input.lower()

In [9]:
#take user input and compare against top 100
def top_recommend(x_song_input):
    if x_song_input in list(top_songs["Song"]):
        ran_num = random.randint(0,99)
        ren_song = top_songs["Song"][ran_num]
        ren_art = top_songs["Singer"][ran_num]
        print("You like {} from the Top Hits chart. You should like {} by {} as well".format(x_song_input.title(), ren_song.title(), ren_art.title()))
    else:
        print("try another song")

In [10]:
top_recommend(song_input())

What's you fauvorite song?baby girl
try another song


In [132]:
#user song feature finder
def sng_feat_finder(x_song_input):
    song = sp.search(q=x_song_input, limit=1,market="GB")
    print("We've found this song {}".format(song["tracks"]["items"][0]["name"]))
    song_id = song["tracks"]["items"][0]["id"]
    song_features = pd.DataFrame(sp.audio_features(song_id))
    song_feature_cut = pd.concat([song_features.iloc[:,0:11], song_features.iloc[:,-1:]], axis=1)
    return np.array(song_feature_cut)

In [135]:
test_song = sng_feat_finder("0V3wPSX9ygBnCm8psDIegu")
def user_song_cluster(x_sng_feat_finder):
    x_sng_feat_finder = x_sng_feat_finder.reshape(1,-1)
    song_trans = scaler2.transform(x_sng_feat_finder)
    cluster = kmeans.predict(song_trans)[0]
    return cluster
user_song_cluster(test_song)

We've found this song A Cappella


/Users/kirilltiufanov/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


3

In [102]:
def get_recommendation(x_user_song_cluster):
    similar_songs = song_dets_clust.loc[song_dets_clust.cluster == x_user_song_cluster]
    random_index = random.randint(0,len(similar_songs))
    song_name, song_id, song_artist,album_name, cluster = song_dets_clust.iloc[random_index]
    return song_name, song_id, song_artist, album_name, cluster 
get_recommendation(3)

('Unforgettable',
 '30f7TNgvsS1AWtyuVqMucH',
 "['Nico Santos']",
 'Streets Of Gold (Unforgettable Edition)',
 4)

In [148]:
def recommender(song = song_input()):
    features = sng_feat_finder(song)
    cluster = user_song_cluster(features)
    song_name, song_id, song_artist, album_name, cluster = get_recommendation(cluster)
    print("We see you likes {}, then {} by {} also should be nice to listen to!".format(song.title(),song_name, song_artist))

recommender()

What's you fauvorite song?califonication
We've found this song Californication
We see you likes Califonication, then Unholy (feat. Kim Petras) by ['Sam Smith', 'Kim Petras'] also should be nice to listen to!


/Users/kirilltiufanov/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
